In [1]:
import string
import pandas as pd

In [2]:
train_file = './data/sr_set-ud-train.conllu'
test_file = './data/sr_set-ud-test.conllu'
dev_file = './data/sr_set-ud-dev.conllu'

In [3]:
lines = []
with open(train_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()
with open(test_file, 'r', encoding='utf-8') as f:
    lines.extend(f.readlines())
with open(dev_file, 'r', encoding='utf-8') as f:
    lines.extend(f.readlines())

In [4]:
open_class_tags = ['ADJ', 'ADV', 'INTJ', 'NOUN', 'PROPN', 'VERB']

In [5]:
ud_content = []
for line in lines:
    if len(line) == 1:
        continue
    if line.startswith("#"):
        continue
    
    parts = line.split('\t') 
    word = parts[1]
    pos_tag = parts[3]
    if pos_tag not in open_class_tags:
        continue
    ud_content.append(word)

In [6]:
len(ud_content)

54632

# Preprocessing

All words are converted to lowercase.

In [7]:
ud_content = [word.lower() for word in ud_content]

Everything that contains digits and punctuation is removed.

In [8]:
ud_content = [word for word in ud_content if all(not c.isdigit() and c not in string.punctuation for c in word)]

In [9]:
data = pd.Series(ud_content)
data_freqs = data.value_counts()
data_freqs[:20]

rekao         332
godine        279
eu            261
takođe        166
bih           151
kada          141
kaže          132
zemlje        131
kosova        128
međutim       126
samo          120
evra          117
još           117
srbije        110
izjavio       107
trebalo       107
predsednik    106
odsto          99
više           98
ministar       94
dtype: int64

# Lexicon structure

Lexicon will be represented as a structure which consists of lemma (the base form), its tag and a list of the derived forms.
Each derived form consits of the form itself and its tag.

In [10]:
class LexiconEntry:
    def __init__(self, lemma, tag):
        self.lemma = lemma
        self.tag = tag
        self.derived_forms = []

In [11]:
lexicon = []

Defining useful functions:

In [12]:
def print_lexicon():
    for lexicon_entry in lexicon:
        lemma = lexicon_entry.lemma
        tag = lexicon_entry.tag

        print("{} [{}]".format(lemma, tag))
        if len(lexicon_entry.derived_forms) == 0:
            continue
        for idx, derived_form_entry in enumerate(lexicon_entry.derived_forms):
            derived_form, derived_form_info = derived_form_entry
            
            if idx == 0:
                print("    ∟ {} {}".format(derived_form, derived_form_info))
            else:
                print("      {} {}".format(derived_form, derived_form_info))

In [13]:
def add_to_lexicon(lemma, base_tag, derived_forms = []):
    global ud_content
    lexicon_entry = LexiconEntry(lemma, base_tag)
    for lex in lexicon:
        if lex.lemma == lemma and lex.tag == base_tag:
            return
    
    ud_content = [word for word in ud_content if word != lemma]

    for der_form in derived_forms:
        lexicon_entry.derived_forms.append(der_form)
    lexicon.append(lexicon_entry)
    
    derived_forms_words = [w[0] for w in derived_forms]
    ud_content = [word for word in ud_content if word not in derived_forms_words]

In [14]:
def get_lexicon_entry(lemma, tag):
    for lex_entry in lexicon:
        if lex_entry.lemma == lemma and lex_entry.tag == tag:
            return lex_entry
        
    return None

Let's start with the most distinctive suffixes, which can be easily be observed

# 1. Verbs

## 1.1 Infinitive

All verbs in Serbian end with: *-ti* or *-ći*

In [15]:
infinitive_verbs_ITI = list(set([w for w in ud_content if w.endswith('iti')]))

In [16]:
infinitive_verbs_ITI

['rušiti',
 'zamisliti',
 'pretpostaviti',
 'raditi',
 'odstupiti',
 'otvoriti',
 'izvršiti',
 'otkriti',
 'iznositi',
 'ograničiti',
 'tražiti',
 'obilaziti',
 'odvratiti',
 'dozvoliti',
 'razrešiti',
 'proceniti',
 'uhvatiti',
 'povrediti',
 'obaviti',
 'izraditi',
 'protiviti',
 'kazniti',
 'štiti',
 'slediti',
 'pružiti',
 'nastaviti',
 'složiti',
 'posvetiti',
 'pridružiti',
 'odrediti',
 'osetiti',
 'razvodniti',
 'obezbediti',
 'uskladiti',
 'vlastiti',
 'otkloniti',
 'proučiti',
 'nastupiti',
 'skladištiti',
 'nadoknaditi',
 'rešiti',
 'uraditi',
 'ohrabriti',
 'zaštititi',
 'odbaciti',
 'uništiti',
 'zanemariti',
 'potrošiti',
 'nuditi',
 'vratiti',
 'ustanoviti',
 'iskoristiti',
 'desiti',
 'završiti',
 'predstaviti',
 'suočiti',
 'opslužiti',
 'zatražiti',
 'izgraditi',
 'ugroziti',
 'napraviti',
 'dobiti',
 'preporučiti',
 'odobriti',
 'proširiti',
 'zaključiti',
 'namestiti',
 'izmeniti',
 'potvrditi',
 'posetiti',
 'podnositi',
 'dogoditi',
 'usredsrediti',
 'voditi',
 'p

In [17]:
for inf_verb in infinitive_verbs_ITI:
    add_to_lexicon(inf_verb, 'VERB')

In [18]:
len(ud_content)

53283

In [19]:
infinitive_verbs_ETI = list(set([w for w in ud_content if w.endswith('eti')]))

In [20]:
infinitive_verbs_ETI

['doneti',
 'početi',
 'aveti',
 'videti',
 'započeti',
 'predvideti',
 'oseti',
 'uspeti',
 'tereti',
 'anketi',
 'uzeti',
 'preneti',
 'fakulteti',
 'našteti',
 'ćančeti',
 'deveti',
 'preduzeti',
 'univerziteti',
 'razumeti',
 'dijeti',
 'peti',
 'oteti',
 'animoziteti',
 'dopreti',
 'posveti',
 'doprineti',
 'predmeti',
 'antikviteti',
 'preoteti',
 'meti',
 'ahmeti',
 'preti',
 'oživeti',
 'poseti',
 'preuzeti',
 'podneti']

In [21]:
for inf_verb in infinitive_verbs_ETI:
    add_to_lexicon(inf_verb, 'VERB')

In [22]:
infinitive_verbs_ATI = list(set([w for w in ud_content if w.endswith('ati')]))

In [23]:
infinitive_verbs_ATI

['menjati',
 'pati',
 'očekivati',
 'izabrati',
 'obavljati',
 'vraćati',
 'pozvati',
 'održavati',
 'omogućavati',
 'hrvati',
 'učestvovati',
 'falsifikovati',
 'prestati',
 'sećati',
 'izdavati',
 'zapošljavati',
 'organizovati',
 'postavljati',
 'aktivirati',
 'plaćati',
 'razvijati',
 'razmenjivati',
 'imati',
 'zadržati',
 'postupati',
 'uticati',
 'sati',
 'preispitati',
 'povećati',
 'preduzimati',
 'klati',
 'insistirati',
 'slati',
 'odavati',
 'oslanjati',
 'advokati',
 'ispričati',
 'reagovati',
 'intervenisati',
 'odvijati',
 'razrešavati',
 'rešavati',
 'uhvati',
 'posmatrati',
 'pokušavati',
 'pregovarati',
 'analizirati',
 'sarađivati',
 'povezati',
 'koštati',
 'držati',
 'funkcionisati',
 'prebacivati',
 'pojačati',
 'pridržavati',
 'glasati',
 'garantovati',
 'pokušati',
 'izazvati',
 'trajati',
 'povrati',
 'dočekivati',
 'bogati',
 'blokirati',
 'sindikati',
 'dati',
 'fokusirati',
 'verovati',
 'rezultati',
 'finansirati',
 'prisustvovati',
 'suspendovati',
 'smatr

In [24]:
for inf_verb in infinitive_verbs_ATI:
    add_to_lexicon(inf_verb, 'VERB')

### 1.2 Past participle

Ending with: *-vši*

In [25]:
past_participle_VSI = list(set([w for w in ud_content if w.endswith('vši')]))
print(past_participle_VSI)

['učinivši', 'zabeleživši', 'dostigavši', 'izazvavši', 'zamenivši', 'pohvalivši', 'naslutivši', 'završivši', 'postavivši', 'pretvorivši', 'uzdrmavši', 'dobivši', 'proširivši', 'bivši', 'rekavši', 'primetivši', 'počevši', 'osvojivši', 'ponovivši', 'pobedivši']


In [26]:
for pp_VSI in past_participle_VSI:
    add_to_lexicon(pp_VSI, 'VERB')

### 1.3 Present participle

Ending with: *-jući*

In [27]:
def add_to_baseform(derived_form, derived_form_info, tag, src_replace, dst_replace):
    for lexicon_entry in lexicon:
        if lexicon_entry.lemma == derived_form.replace(src_replace, dst_replace) and lexicon_entry.tag == tag:
            lexicon_entry.derived_forms.append((derived_form, derived_form_info))
            return
    
    lemma = derived_form.replace(src_replace, dst_replace)
    derived_forms = [(derived_form, derived_form_info)]
    add_to_lexicon(lemma, tag, derived_forms)

In [28]:
present_participle_JUCI = list(set([w for w in ud_content if w.endswith('jući')]))

for pp in present_participle_JUCI:
    if pp.endswith('ajući'):
        add_to_baseform(pp, "PRESENT_PAT", "VERB", "ajući", "ati")

    elif pp.endswith('ujući'):
        add_to_baseform(pp, "PRESENT_PAT", "VERB", "ujući", "ovati")

In [29]:
present_participle_JUCI

['nazivajući',
 'znajući',
 'izdvajajući',
 'čitajući',
 'uzrokujući',
 'predsedavajući',
 'omogućavajući',
 'zahvaljujući',
 'dajući',
 'otvarajući',
 'obeležavajući',
 'završavajući',
 'preneražavajući',
 'napominjući',
 'pojavljujući',
 'obećavajući',
 'nedostajući',
 'preduzimajući',
 'primoravajući',
 'potiskujući',
 'prezajući',
 'zbunjujući',
 'zbližavajući',
 'zatvarajući',
 'tragajući',
 'opisujući',
 'favorizujući',
 'oslobađajući',
 'ažurirajući',
 'otuđujući',
 'sprečavajući',
 'optužujući',
 'upozoravajući',
 'igrajući',
 'obraćajući',
 'ponavljajući',
 'uključujući',
 'pozivajući',
 'savetujući',
 'najavljujući',
 'okončavajući',
 'odbijajući',
 'naglašavajući',
 'pozdravljajući',
 'dodajući',
 'bacajući',
 'suočavajući',
 'zahtevajući',
 'diskreditujući',
 'prodajući',
 'saopštavajući',
 'opstruirajući',
 'podsećajući',
 'predstavljajući',
 'ubacujući',
 'uveravajući',
 'emigrirajući',
 'imajući',
 'postavljajući',
 'insistirajući',
 'kritikujući',
 'narastajući',
 'ukaz

Another type of suffix of present participle form is *-eći*, but it can be mixed with infinitive form.

In [30]:
suffix_ECI = list(set([w for w in ud_content if w.endswith('eći')]))

In [31]:
suffix_ECI

['izlazeći',
 'veći',
 'tražeći',
 'paleći',
 'tvrdeći',
 'donoseći',
 'steći',
 'najveći',
 'odlazeći',
 'sudeći',
 'navodeći',
 'doprinoseći',
 'sledeći',
 'baveći',
 'preći',
 'važeći',
 'deleći',
 'kršeći',
 'postojeći',
 'predstojeći',
 'beležeći',
 'govoreći',
 'nesreći',
 'reći',
 'preseći',
 'koristeći',
 'trudeći',
 'poreći',
 'misleći',
 'treći']

# 2. Adjectives

## 2.1 Comparative

Suffixes:
- Singular: *-niji (m.), -nija (f.), -nije (n.)*
- Plural: *-nijim

In [32]:
comparative_forms = [w for w in ud_content if w.endswith('niji') or w.endswith('nija') or w.endswith('nije') or w.endswith('nijim')]
comparative_forms = [w for w in comparative_forms if not w.startswith('naj')]
comparative_forms = list(set(comparative_forms))

In [33]:
comparative_forms

['sloveniji',
 'beskompromisniji',
 'uticajniji',
 'linija',
 'kompleksnija',
 'kompaniji',
 'estonije',
 'slovenije',
 'opasnijim',
 'važnija',
 'čečeniji',
 'decenije',
 'očigledniji',
 'transparentnijim',
 'tačnije',
 'španiji',
 'urgentniji',
 'docnije',
 'aktivnije',
 'makedonija',
 'uniji',
 'britanija',
 'precizniji',
 'alianija',
 'makedonije',
 'rumuniji',
 'represivniji',
 'ceremoniji',
 'litvanije',
 'apolonija',
 'slobodniji',
 'unije',
 'značajnije',
 'važnije',
 'jeftinije',
 'demokratičnija',
 'konkurentnija',
 'svestranije',
 'ranijim',
 'važniji',
 'unija',
 'sigurnije',
 'letonije',
 'potpunije',
 'makedoniji',
 'menija',
 'pensilvaniji',
 'rumunija',
 'ozbiljniji',
 'ranije',
 'britanije',
 'efikasniji',
 'zainteresovanija',
 'rasprostranjenija',
 'albanije',
 'kompanije',
 'bezbedniji',
 'pozitivniji',
 'konkretnije',
 'efikasnija',
 'britaniji',
 'jevtinijim',
 'decenija',
 'ceremonije',
 'kompanija',
 'snažnije',
 'optimalnije',
 'albanija',
 'sigurnija',
 'izraže

We will build positive forms of the comparative forms:

-niji $\rightarrow$ -an

-nija $\rightarrow$ -na

-nije $\rightarrow$ -no

-nijim $\rightarrow$ -an 
    - there's no way of knowing if the comparative form is supposed to be in masculine, feminine or neutral form, so here it is deliberately chosen that the masculine form will be the base form

In [34]:
for comparative in comparative_forms:
    positive_lexicon_form = None
    if comparative.endswith("niji"):
        positive = comparative.replace("niji", "an")
        positive_lexicon_form = (comparative, 'COMP-M-SG')
    if comparative.endswith("nija"):
        positive = comparative.replace("nija", "na")
        positive_lexicon_form = (comparative, 'COMP-F-SG')
    if comparative.endswith("nije"):
        positive = comparative.replace("nije", "no")
        positive_lexicon_form = (comparative, 'COMP-N-SG')
    if comparative.endswith("nijim"):
        positive = comparative.replace("nijim", "an")
        positive_lexicon_form = (comparative, 'COMP-M-PL')
    
    add_to_lexicon(positive, 'ADJ', [positive_lexicon_form])

In [35]:
print_lexicon()

rušiti [VERB]
zamisliti [VERB]
pretpostaviti [VERB]
raditi [VERB]
odstupiti [VERB]
otvoriti [VERB]
izvršiti [VERB]
otkriti [VERB]
iznositi [VERB]
ograničiti [VERB]
tražiti [VERB]
obilaziti [VERB]
odvratiti [VERB]
dozvoliti [VERB]
razrešiti [VERB]
proceniti [VERB]
uhvatiti [VERB]
povrediti [VERB]
obaviti [VERB]
izraditi [VERB]
protiviti [VERB]
kazniti [VERB]
štiti [VERB]
slediti [VERB]
pružiti [VERB]
nastaviti [VERB]
složiti [VERB]
posvetiti [VERB]
pridružiti [VERB]
odrediti [VERB]
osetiti [VERB]
razvodniti [VERB]
obezbediti [VERB]
uskladiti [VERB]
vlastiti [VERB]
otkloniti [VERB]
proučiti [VERB]
nastupiti [VERB]
skladištiti [VERB]
nadoknaditi [VERB]
rešiti [VERB]
uraditi [VERB]
ohrabriti [VERB]
zaštititi [VERB]
odbaciti [VERB]
uništiti [VERB]
zanemariti [VERB]
potrošiti [VERB]
nuditi [VERB]
vratiti [VERB]
ustanoviti [VERB]
iskoristiti [VERB]
desiti [VERB]
završiti [VERB]
predstaviti [VERB]
suočiti [VERB]
opslužiti [VERB]
zatražiti [VERB]
izgraditi [VERB]
ugroziti [VERB]
napraviti [VERB

## 2.2 Superlative

It is fairly easy to find adjectives in superlative form because they almost always have the prefix *naj-*.

In [36]:
prefix_NAJ = list(set([w for w in ud_content if w.startswith('naj')]))

In [37]:
prefix_NAJ

['najbogatijih',
 'najneslobodnija',
 'najčešće',
 'najčuvanijih',
 'najugledniji',
 'najdelikatnijoj',
 'najbolja',
 'najtraženija',
 'najvećeg',
 'najjačih',
 'najboljem',
 'najduži',
 'najavljuje',
 'najraznovrsnije',
 'najgorim',
 'najznačajniji',
 'najesen',
 'najrazličitijih',
 'najzad',
 'najnoviji',
 'najbezbednija',
 'najavile',
 'najizolovanijom',
 'najgore',
 'najvažnijih',
 'najozbiljnija',
 'najurgentniji',
 'najtiražnijeg',
 'najavljena',
 'najvećem',
 'najmodernijeg',
 'najnoviju',
 'najteža',
 'najnovijem',
 'najjednostavniji',
 'najverovatnije',
 'najveći',
 'najrelevantnijih',
 'najvećih',
 'najmoćniji',
 'najpouzdniji',
 'najniži',
 'najtalentovanijih',
 'najmanju',
 'najstarijim',
 'najisplativija',
 'najuglednijih',
 'najbezbednijim',
 'najnasilniju',
 'najprimetnijih',
 'najpopularnija',
 'najznačajnijih',
 'najčuvenijih',
 'najvažniju',
 'najizolovanijih',
 'najavio',
 'najbolje',
 'najrashlađenija',
 'najslobodnija',
 'najznačajnija',
 'najkraća',
 'najvažnijim'

In [38]:
def is_superlative_of_existing_base_form(lexicon_entry, superlative):
    if len(lexicon_entry.derived_forms) == 0:
        return False
    
    comparative = superlative[3:]
    for d_form in lexicon_entry.derived_forms:
        if d_form[1].startswith("COMP-") and d_form[0].startswith(comparative):
            gender = d_form[1].split("-")[1]
            number = d_form[1].split("-")[2]
            lexicon_entry.derived_forms.append((superlative, "SUP-{}-{}".format(gender, number)))
            return True
        
    return False
            

for p_NAJ in prefix_NAJ:
    lexicon_entries = [l for l in lexicon if is_superlative_of_existing_base_form(l, p_NAJ)]

In [39]:
print_lexicon()

rušiti [VERB]
zamisliti [VERB]
pretpostaviti [VERB]
raditi [VERB]
odstupiti [VERB]
otvoriti [VERB]
izvršiti [VERB]
otkriti [VERB]
iznositi [VERB]
ograničiti [VERB]
tražiti [VERB]
obilaziti [VERB]
odvratiti [VERB]
dozvoliti [VERB]
razrešiti [VERB]
proceniti [VERB]
uhvatiti [VERB]
povrediti [VERB]
obaviti [VERB]
izraditi [VERB]
protiviti [VERB]
kazniti [VERB]
štiti [VERB]
slediti [VERB]
pružiti [VERB]
nastaviti [VERB]
složiti [VERB]
posvetiti [VERB]
pridružiti [VERB]
odrediti [VERB]
osetiti [VERB]
razvodniti [VERB]
obezbediti [VERB]
uskladiti [VERB]
vlastiti [VERB]
otkloniti [VERB]
proučiti [VERB]
nastupiti [VERB]
skladištiti [VERB]
nadoknaditi [VERB]
rešiti [VERB]
uraditi [VERB]
ohrabriti [VERB]
zaštititi [VERB]
odbaciti [VERB]
uništiti [VERB]
zanemariti [VERB]
potrošiti [VERB]
nuditi [VERB]
vratiti [VERB]
ustanoviti [VERB]
iskoristiti [VERB]
desiti [VERB]
završiti [VERB]
predstaviti [VERB]
suočiti [VERB]
opslužiti [VERB]
zatražiti [VERB]
izgraditi [VERB]
ugroziti [VERB]
napraviti [VERB

- Prisvojni pridevi (mesta): -ski, -čki, -ški
- Prisvojni pridevi (?): -ni
- Prisvojni pridevi (ljudi): -ov, -in, -ev

## 2.3 Possessive adjectives

Possessive adjectives can be of the following forms depending on the type of entity that *possesses*:

1. *-ov, -ev, -in*: when the entity that possesses is a human being
    - Examples: Markov "Mark's", Tonijev "Tony's", Nemanjin "Nemanja's" 
2. *-ski, -čki, -ški*: usually with toponymical entities - countries, ethnicities, cities, villages, regions, etc.
    - Examples: srpski "Serbian", nemački "German", praški "Prague's"
3. *-ni*
    - Examples: državni "state"

Of course, adjectives are changed according to gender as well, so we have:

2. *-ska* (f.), *-sko* (n.), *-čka* (f.), *-čko (n.), *-ška* (f.), *-ško* (n.)
3. *-na* (f.), *-no* (n.)

However, the inflection forms for masculine, feminine and neutral gender are the same when inflecting over different cases. The only difference is in the nominative form, so feminine and neutral forms are put under the masculine form.

Singular

| case | masculine | feminine | neutral | 
|------|-----------|----------|---------|
|nominative|srpski|srpska|srpsko|
|genitive|srpskog|srpske|srpskog|
|dative|srpskom|srpskoj|srpskom|
|acusative|srpski|srpsku|srpsko|
|vocative|srpski|srpska|srpsko|
|instrumental|srpskim|srpskom|srpskim|
|locative|srpskom|srpskoj|srpskom|

Plural

| case | masculine | feminine | neutral | 
|------|-----------|----------|---------|
|nominative|srpski|srpske|srpska|
|genitive|srpskih|srpskih|srpske|
|dative|srpskim|srpskim|srpskoj|
|acusative|srpske|srpske|srpsku|
|vocative|srpski|srpske|srpska|
|instrumental|srpskim|srpskim|srpskom|
|locative|srpskim|srpskim|srpskoj|

Possible siffixes: *-ski*, *-skog*, *-skom*, *-skim*, *-ska*, *-ske*, *-skoj*, *-sku*, *-skih*, *-sko*

### 2.3.1 suffix *-ski* and its inflections

In [40]:
possessive_adjectives = list(set([w for w in ud_content if w.endswith('ski')]))

In [41]:
possessive_adjectives

['rumunski',
 'španski',
 'vremenski',
 'gagovski',
 'konjanovski',
 'teoretski',
 'švedski',
 'tetovski',
 'gruevski',
 'irski',
 'makedonski',
 'trajkovski',
 'imunski',
 'diplomatski',
 'hrvatski',
 'bugarski',
 'meteorski',
 'novinarski',
 'ivanovski',
 'evropski',
 'romski',
 'zemunski',
 'peruanski',
 'saudijski',
 'šahovski',
 'prevarantski',
 'vijetnamski',
 'ministarski',
 'turski',
 'amaterski',
 'londonski',
 'sofijski',
 'aleksievski',
 'briselski',
 'verski',
 'nikolovski',
 'balkanski',
 'republikanski',
 'lavovski',
 'belgijski',
 'beogradski',
 'gruzijski',
 'peševski',
 'kablovski',
 'automatski',
 'kosovski',
 'jugoslovenski',
 'evroatlantski',
 'trojanski',
 'univerzitetski',
 'holivudski',
 'portugalski',
 'puninski',
 'penzijski',
 'antonovski',
 'britanski',
 'poposki',
 'orvelovski',
 'ohridski',
 'ljudski',
 'levičarski',
 'finski',
 'istorijski',
 'izraelski',
 'kurdski',
 'avionski',
 'vrhunski',
 'svetski',
 'moldavski',
 'posttraumatski',
 'litvanski',
 'alb

In [42]:
for possesive_adj in possessive_adjectives:
    add_to_lexicon(possesive_adj, "ADJ")

In [43]:
def add_possessive_adjectives(possessive_adjectives, replace_src, replace_dst):
    for possessive_ADJ in possessive_adjectives:
        derived_f = (possessive_ADJ, "POSSESSIVE")
        base_form = possessive_ADJ.replace(replace_src, replace_dst)
        lexicon_entry = get_lexicon_entry(base_form, "ADJ")
        if lexicon_entry is not None:
            lexicon_entry.derived_forms.append(derived_f)
        else:
            add_to_lexicon(base_form, "ADJ", [derived_f])

In [44]:
def add_adjective_inflection_forms(inflection_suffixes):
    for suffix, base_suffix in inflection_suffixes:
        possessive_adjectives = list(set([w for w in ud_content if w.endswith(suffix)]))
        add_possessive_adjectives(possessive_adjectives, suffix, base_suffix)

In [45]:
inflection_suffixes = [('skog', 'ski'), ('skom', 'ski'), ('skim', 'ski'), ('ska', 'ski'), ('ske', 'ski'), 
                       ('skoj', 'ski'), ('sku', 'ski'), ('skih', 'ski'), ('sko', 'ski')]
add_adjective_inflection_forms(inflection_suffixes)

In [46]:
print_lexicon()

rušiti [VERB]
zamisliti [VERB]
pretpostaviti [VERB]
raditi [VERB]
odstupiti [VERB]
otvoriti [VERB]
izvršiti [VERB]
otkriti [VERB]
iznositi [VERB]
ograničiti [VERB]
tražiti [VERB]
obilaziti [VERB]
odvratiti [VERB]
dozvoliti [VERB]
razrešiti [VERB]
proceniti [VERB]
uhvatiti [VERB]
povrediti [VERB]
obaviti [VERB]
izraditi [VERB]
protiviti [VERB]
kazniti [VERB]
štiti [VERB]
slediti [VERB]
pružiti [VERB]
nastaviti [VERB]
složiti [VERB]
posvetiti [VERB]
pridružiti [VERB]
odrediti [VERB]
osetiti [VERB]
razvodniti [VERB]
obezbediti [VERB]
uskladiti [VERB]
vlastiti [VERB]
otkloniti [VERB]
proučiti [VERB]
nastupiti [VERB]
skladištiti [VERB]
nadoknaditi [VERB]
rešiti [VERB]
uraditi [VERB]
ohrabriti [VERB]
zaštititi [VERB]
odbaciti [VERB]
uništiti [VERB]
zanemariti [VERB]
potrošiti [VERB]
nuditi [VERB]
vratiti [VERB]
ustanoviti [VERB]
iskoristiti [VERB]
desiti [VERB]
završiti [VERB]
predstaviti [VERB]
suočiti [VERB]
opslužiti [VERB]
zatražiti [VERB]
izgraditi [VERB]
ugroziti [VERB]
napraviti [VERB

### 2.3.2 suffix *-čki* and its inflections


Singular

| case | masculine | feminine | neutral | 
|------|-----------|----------|---------|
|nominative|nemački|nemačka|nemačko|
|genitive|nemačkog|nemačke|nemačkog|
|dative|nemačkom|nemačkoj|nemačkom|
|acusative|nemački|nemačku|nemačko|
|vocative|nemački|nemačka|nemačko|
|instrumental|nemačkim|nemačkom|nemačkim|
|locative|nemačkom|nemačkoj|nemačkom|

Plural

| case | masculine | feminine | neutral | 
|------|-----------|----------|---------|
|nominative|nemački|nemačke|nemačka|
|genitive|nemačkih|nemačkih|nemačke|
|dative|nemačkim|nemačkim|nemačkoj|
|acusative|nemačke|nemačke|nemačku|
|vocative|nemački|nemačke|nemačka|
|instrumental|nemačkim|nemačkim|nemačkom|
|locative|nemačkim|nemačkim|nemačkoj|

Possible suffixes: *-čki*, *-čkog*, *-čkom*, *-čka*, *-čkih*, *-čkim*, *-čko*, *-čke*, *-čkoj*, *-čku*

In [47]:
possessive_adjectives = list(set([w for w in ud_content if w.endswith('čki')]))

In [48]:
possessive_adjectives

['predsednički',
 'bošnjački',
 'železnički',
 'američki',
 'muzički',
 'radnički',
 'tehnički',
 'umetnički',
 'putnički',
 'ekstremistički',
 'teroristički',
 'etnički',
 'monopolistički',
 'statistički',
 'posmatrački',
 'preduzetnički',
 'politički',
 'slovenački',
 'turistički',
 'višestranački',
 'pacifički',
 'stranački',
 'grčki',
 'komunistički',
 'irački',
 'populistički',
 'brodovlasnički',
 'nemački',
 'prostački',
 'zajednički',
 'pregovarački',
 'međuetnički',
 'ultranacionalistički']

In [49]:
for possesive_adj in possessive_adjectives:
    add_to_lexicon(possesive_adj, "ADJ")

In [50]:
inflection_suffixes = [('čkog', 'čki'), ('čkom', 'čki'), ('čka', 'čki'), ('čkih', 'čki'), ('čkim', 'čki'), ('čko', 'čki'), 
                      ('čke', 'čki'), ('čkoj', 'čki'), ('čku', 'čki')]

add_adjective_inflection_forms(inflection_suffixes)

In [51]:
print_lexicon()

rušiti [VERB]
zamisliti [VERB]
pretpostaviti [VERB]
raditi [VERB]
odstupiti [VERB]
otvoriti [VERB]
izvršiti [VERB]
otkriti [VERB]
iznositi [VERB]
ograničiti [VERB]
tražiti [VERB]
obilaziti [VERB]
odvratiti [VERB]
dozvoliti [VERB]
razrešiti [VERB]
proceniti [VERB]
uhvatiti [VERB]
povrediti [VERB]
obaviti [VERB]
izraditi [VERB]
protiviti [VERB]
kazniti [VERB]
štiti [VERB]
slediti [VERB]
pružiti [VERB]
nastaviti [VERB]
složiti [VERB]
posvetiti [VERB]
pridružiti [VERB]
odrediti [VERB]
osetiti [VERB]
razvodniti [VERB]
obezbediti [VERB]
uskladiti [VERB]
vlastiti [VERB]
otkloniti [VERB]
proučiti [VERB]
nastupiti [VERB]
skladištiti [VERB]
nadoknaditi [VERB]
rešiti [VERB]
uraditi [VERB]
ohrabriti [VERB]
zaštititi [VERB]
odbaciti [VERB]
uništiti [VERB]
zanemariti [VERB]
potrošiti [VERB]
nuditi [VERB]
vratiti [VERB]
ustanoviti [VERB]
iskoristiti [VERB]
desiti [VERB]
završiti [VERB]
predstaviti [VERB]
suočiti [VERB]
opslužiti [VERB]
zatražiti [VERB]
izgraditi [VERB]
ugroziti [VERB]
napraviti [VERB

### 2.3.3 suffix *-ški* and its inflections


Singular

| case | masculine | feminine | neutral | 
|------|-----------|----------|---------|
|nominative|češki|češka|češko|
|genitive|češkog|češke|češkog|
|dative|češkom|češkoj|češkom|
|acusative|češki|češku|češko|
|vocative|češki|češka|češko|
|instrumental|češkim|češkom|češkim|
|locative|češkom|češkoj|češkom|

Plural

| case | masculine | feminine | neutral | 
|------|-----------|----------|---------|
|nominative|češki|češke|češka|
|genitive|čeških|čeških|češke|
|dative|češkim|češkim|češkoj|
|acusative|češke|češke|češku|
|vocative|češki|češke|češka|
|instrumental|češkim|češkim|češkom|
|locative|češkim|češkim|češkoj|

Possible suffixes: *-ški*, *-škog*, *-škom*, *-ška*, *-ških*, *-škim*, *-ško*, *-ške*, *-škoj*, *-šku*

In [52]:
possessive_adjectives = list(set([w for w in ud_content if w.endswith('ški')]))

In [53]:
possessive_adjectives

['zoološki',
 'ideološki',
 'haški',
 'malteški',
 'muški',
 'tehnološki',
 'bombaški',
 'norveški',
 'češki',
 'njujorški',
 'strateški',
 'luksemburški']

In [54]:
for possesive_adj in possessive_adjectives:
    add_to_lexicon(possesive_adj, "ADJ")

In [55]:
inflection_suffixes = [('škog', 'ški'), ('škom', 'ški'), ('ška', 'ški'), ('ških', 'ški'), ('škim', 'ški'), ('ško', 'ški'), 
                      ('ške', 'ški'), ('škoj', 'ški'), ('šku', 'ški')]

add_adjective_inflection_forms(inflection_suffixes)

In [56]:
print_lexicon()

rušiti [VERB]
zamisliti [VERB]
pretpostaviti [VERB]
raditi [VERB]
odstupiti [VERB]
otvoriti [VERB]
izvršiti [VERB]
otkriti [VERB]
iznositi [VERB]
ograničiti [VERB]
tražiti [VERB]
obilaziti [VERB]
odvratiti [VERB]
dozvoliti [VERB]
razrešiti [VERB]
proceniti [VERB]
uhvatiti [VERB]
povrediti [VERB]
obaviti [VERB]
izraditi [VERB]
protiviti [VERB]
kazniti [VERB]
štiti [VERB]
slediti [VERB]
pružiti [VERB]
nastaviti [VERB]
složiti [VERB]
posvetiti [VERB]
pridružiti [VERB]
odrediti [VERB]
osetiti [VERB]
razvodniti [VERB]
obezbediti [VERB]
uskladiti [VERB]
vlastiti [VERB]
otkloniti [VERB]
proučiti [VERB]
nastupiti [VERB]
skladištiti [VERB]
nadoknaditi [VERB]
rešiti [VERB]
uraditi [VERB]
ohrabriti [VERB]
zaštititi [VERB]
odbaciti [VERB]
uništiti [VERB]
zanemariti [VERB]
potrošiti [VERB]
nuditi [VERB]
vratiti [VERB]
ustanoviti [VERB]
iskoristiti [VERB]
desiti [VERB]
završiti [VERB]
predstaviti [VERB]
suočiti [VERB]
opslužiti [VERB]
zatražiti [VERB]
izgraditi [VERB]
ugroziti [VERB]
napraviti [VERB